In [1]:
import pandas as pd
import numpy as np
import re
from cuts import cosmo_cuts

import warnings
warnings.filterwarnings(action='once')
pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 600)

In [2]:
mb = lambda x0: -2.5 * np.log10(x0) + 10.635
mberr = lambda x0, x0err: 2.5 * (x0err / x0) / np.log(10)

Reading in Data

In [3]:
master = pd.read_csv('data/master_v3.1.csv')
master['mb_snana'] = mb(master['x0_snana'])
master['mb_err_snana'] = mberr(master['x0_snana'],master['x0_err_snana'])
master['mb_sncosmo'] = mb(master['x0_sncosmo'])
master['mb_err_sncosmo'] = mberr(master['x0_sncosmo'],master['x0_err_sncosmo'])
master.sort_values(by='Name',inplace=True)
master.head()

,Unnamed: 0,Unnamed: 0.1,Name,z_burns2018_bvriC,z_burns2018_griC,EBV,EBV_err,dm15,dm15_err,MV,MV_err,bv_source_w09,Type,foundation,csp,w09,vel,vel_err,vel_source,bv_source,z,z_err,z_wang09,c_sncosmo,c_err_sncosmo,chisq,mwebv_sncosmo,mwebv_err,success,t0,t0_err,x0_sncosmo,x0_err_sncosmo,x1_sncosmo,x1_err_sncosmo,z_cosana,Name_sncosmo,Survey_sncosmo,bands,bands2,Name_upper,Name_snana,zHEL,zCMB,zHD,mwebv_snana,SNRMAX1,SNRMAX2,PKMJD,PKMJDERR,x1_snana,x1_err_snana,c_snana,c_err_snana,mB,mBERR,x0_snana,x0_err_snana,FITCHI2,x0_offset,Survey_snana,first_fitmjd,mb_snana,mb_err_snana,mb_sncosmo,mb_err_sncosmo
104,104,125,1981b,0.00603,0.00603,0.147,0.023,1.14,0.062,-18.96,0.007,True,HV,False,False,True,-99.9,NaN,w09,b18,0.003933,NaN,0.003933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,303,506,1983g,NaN,NaN,0.230,0.090,1.30,0.100,-18.90,0.450,True,HV,False,False,True,-99.9,NaN,w09,w09,0.004993,NaN,0.004993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,304,507,1984a,NaN,NaN,0.260,0.080,1.20,0.100,-18.91,0.570,True,HV,False,False,True,-99.9,NaN,w09,w09,0.003933,NaN,0.003933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,305,508,1989a,NaN,NaN,0.230,0.090,1.05,0.100,-19.17,0.250,True,HV,False,False,True,-99.9,NaN,w09,w09,0.009193,NaN,0.009193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,296,499,1989b,NaN,NaN,0.390,0.050,1.35,0.050,-18.15,0.150,True,N,False,False,True,99.9,NaN,w09,w09,0.001831,NaN,0.001831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
massvel = pd.read_csv('data/mass_vel_data_expanded_v1.1.dat')
massvel = massvel[['Name','mass','masserr','vel','velerr','subsample','symbol']]
massvel.replace(to_replace='asassn1',value='asassn-1',regex=True,inplace=True)
massvel.head()

,Name,mass,masserr,vel,velerr,subsample,symbol
0,2006ot,11.361939,0.158451,NaN,NaN,CSP,o
1,asassn-15il,10.134000,0.158451,-10.834621,0.220574,Foundation,o
2,asassn-15mg,10.134000,0.158451,-10.650190,0.229880,Foundation,o
3,ps15coh,8.630000,0.158451,-10.778920,0.220770,Foundation,o
4,asassn-15sf,9.365000,0.158451,-9.868772,0.220276,Foundation,o


In [5]:
found_cosmo = pd.read_csv('data/salt2mu_kyle_foundsn.fitres', comment='#', delim_whitespace=True)
found_cosmo.rename(columns={'CID':'Name'},inplace=True)
found_cosmo.Name = found_cosmo.Name.str.lower()
found_cosmo.head()

,VARNAMES:,Name,IDSURVEY,TYPE,FIELD,CUTFLAG_SNANA,zHEL,zHELERR,zCMB,zCMBERR,zHD,zHDERR,VPEC,VPECERR,MWEBV,HOST_LOGMASS,HOST_LOGMASS_ERR,HOST_sSFR,HOST_sSFR_ERR,SNRMAX1,SNRMAX2,SNRMAX3,PKMJD,PKMJDERR,x1,x1ERR,c,cERR,mB,mBERR,x0,x0ERR,COV_x1_c,COV_x1_x0,COV_c_x0,NDOF,FITCHI2,FITPROB,CUTMASK,MU,MUMODEL,MUERR,MUERR_RAW,MURES,MUPULL,M0DIF,CHI2,biasCor_mu,biasCorErr_mu,biasCor_mB,biasCor_x1,biasCor_c,biasScale_muCOV,IDSAMPLE
0,SN:,2016cvv,150,1,NaN,1,0.04457,0.001,0.04457,0.001,0.04457,0.001,0,0,0.088616,10,10,-9,-9,64.6468,47.7019,-99.0,57558.3,0.623,1.36625,0.28502,0.067344,0.038454,17.2094,0.048920,0.002346,0.000106,-0.002162,4.698650e-06,-0.000003,8,26.2064,0.000968,0,36.4811,36.4777,0.1655,0.1090,-0.0042,-0.025,0.0076,-0.18,-0.0304,0.009,-0.0084,0.088,0.0107,0.968,0
1,SN:,asassn-16aj,150,1,NaN,1,0.03074,0.001,0.03074,0.001,0.03074,0.001,0,0,0.028175,10,10,-9,-9,213.1830,88.4127,-99.0,57402.0,0.374,-1.76042,0.14274,-0.017720,0.033384,16.4121,0.041258,0.004888,0.000186,-0.000070,1.469140e-06,-0.000006,18,13.0884,0.786300,0,35.5415,35.6486,0.1619,0.1011,-0.1030,-0.636,-0.0041,0.21,-0.0263,0.005,-0.0127,-0.026,0.0033,0.983,0
2,SN:,asassn-15mg,150,1,NaN,1,0.04278,0.001,0.04278,0.001,0.04278,0.001,0,0,0.027882,10,10,-9,-9,69.8776,63.3984,-99.0,57218.1,0.133,-1.55052,0.15105,-0.031991,0.034873,17.2190,0.037616,0.002325,0.000081,-0.000605,-7.774330e-07,-0.000002,13,11.4682,0.571660,0,36.4241,36.3858,0.1602,0.0992,0.0307,0.192,0.0076,-0.16,-0.0307,0.005,-0.0202,-0.024,0.0024,0.975,0
3,SN:,css160129,150,1,NaN,1,0.06766,0.001,0.06766,0.001,0.06766,0.001,0,0,0.032471,10,10,-9,-9,60.9497,59.2371,-99.0,57416.4,0.601,0.59887,0.16766,-0.148981,0.035385,17.5212,0.043008,0.001760,0.000070,-0.002033,3.853730e-06,-0.000002,12,17.4630,0.133000,0,37.2831,37.4204,0.1560,0.0935,-0.0630,-0.404,-0.0744,-0.04,0.0529,0.011,-0.0691,0.001,-0.0392,0.966,0
4,SN:,asassn-15pn,150,1,NaN,1,0.03836,0.001,0.03836,0.001,0.03836,0.001,0,0,0.081047,10,10,-9,-9,75.7503,63.6262,-99.0,57280.4,0.232,0.42120,0.15637,0.011422,0.033358,16.9392,0.040062,0.003008,0.000111,-0.000191,-1.813250e-06,-0.000003,10,17.4728,0.064535,0,36.2625,36.1419,0.1592,0.0971,0.0314,0.197,0.0893,-0.16,-0.0299,0.004,-0.0124,-0.010,0.0052,0.979,0


Combining Datasets

In [6]:
master_mass = pd.merge(master, massvel, on='Name', how='outer', suffixes=('','_massvel'))
master_mass.head()

,Unnamed: 0,Unnamed: 0.1,Name,z_burns2018_bvriC,z_burns2018_griC,EBV,EBV_err,dm15,dm15_err,MV,MV_err,bv_source_w09,Type,foundation,csp,w09,vel,vel_err,vel_source,bv_source,z,z_err,z_wang09,c_sncosmo,c_err_sncosmo,chisq,mwebv_sncosmo,mwebv_err,success,t0,t0_err,x0_sncosmo,x0_err_sncosmo,x1_sncosmo,x1_err_sncosmo,z_cosana,Name_sncosmo,Survey_sncosmo,bands,bands2,Name_upper,Name_snana,zHEL,zCMB,zHD,mwebv_snana,SNRMAX1,SNRMAX2,PKMJD,PKMJDERR,x1_snana,x1_err_snana,c_snana,c_err_snana,mB,mBERR,x0_snana,x0_err_snana,FITCHI2,x0_offset,Survey_snana,first_fitmjd,mb_snana,mb_err_snana,mb_sncosmo,mb_err_sncosmo,mass,masserr,vel_massvel,velerr,subsample,symbol
0,104.0,125.0,1981b,0.00603,0.00603,0.147,0.023,1.14,0.062,-18.96,0.007,True,HV,False,False,True,-99.9,NaN,w09,b18,0.003933,NaN,0.003933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.4298,0.126554,-99.9,NaN,old,o
1,303.0,506.0,1983g,NaN,NaN,0.230,0.090,1.30,0.100,-18.90,0.450,True,HV,False,False,True,-99.9,NaN,w09,w09,0.004993,NaN,0.004993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,304.0,507.0,1984a,NaN,NaN,0.260,0.080,1.20,0.100,-18.91,0.570,True,HV,False,False,True,-99.9,NaN,w09,w09,0.003933,NaN,0.003933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,305.0,508.0,1989a,NaN,NaN,0.230,0.090,1.05,0.100,-19.17,0.250,True,HV,False,False,True,-99.9,NaN,w09,w09,0.009193,NaN,0.009193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,296.0,499.0,1989b,NaN,NaN,0.390,0.050,1.35,0.050,-18.15,0.150,True,N,False,False,True,99.9,NaN,w09,w09,0.001831,NaN,0.001831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
master_mass.loc[master_mass[np.isnan(master_mass.vel)].index,'vel'] = master_mass[np.isnan(master_mass.vel)].vel_massvel
master_mass.loc[master_mass[np.isnan(master_mass.vel_err)].index,'vel_err'] = master_mass[np.isnan(master_mass.vel)].velerr

In [8]:
for i in master_mass.index:
    # Removing objects that don't have a good velocity measurement, a mass measurement, or a lc fit
    if (np.abs(master_mass.loc[i,'vel']) > 90) and (np.isnan(master_mass.loc[i,'mass'])) and (np.isnan(master_mass.loc[i,'c_snana'])):
        master_mass.drop(i,inplace=True)
    # Removing objects that have a mass measurement, but don't have a good velocity or a lc fit
    elif (~np.isnan(master_mass.loc[i,'mass'])) and (np.abs(master_mass.loc[i,'vel']) > 90) and (np.isnan(master_mass.loc[i,'c_snana'])):
        master_mass.drop(i,inplace=True)
    # Removing objects that don't have a mass measurement or a lc fit
    elif (np.isnan(master_mass.loc[i,'mass'])) and (np.isnan(master_mass.loc[i,'c_snana'])):
        master_mass.drop(i,inplace=True)

In [9]:
master_found_cosmo = pd.merge(master_mass[master_mass.foundation == True], 
                              found_cosmo[['Name','MU','MURES','MUERR']], 
                              on='Name', how='outer')
master_found_cosmo.sort_values(by='Name',inplace=True)

In [10]:
for i in master_found_cosmo.index:
    if (np.abs(master_found_cosmo.loc[i,'vel']) > 90) and (np.isnan(master_found_cosmo.loc[i,'mass'])) and (np.isnan(master_found_cosmo.loc[i,'c_snana'])):
        master_found_cosmo.drop(i,inplace=True)
    elif (~np.isnan(master_found_cosmo.loc[i,'mass'])) and (np.abs(master_found_cosmo.loc[i,'vel']) > 90) and (np.isnan(master_found_cosmo.loc[i,'c_snana'])):
        master_found_cosmo.drop(i,inplace=True)
    elif (np.isnan(master_found_cosmo.loc[i,'mass'])) and (np.isnan(master_found_cosmo.loc[i,'c_snana'])):
        master_found_cosmo.drop(i,inplace=True)

In [11]:
master_found_cosmo.head()

,Unnamed: 0,Unnamed: 0.1,Name,z_burns2018_bvriC,z_burns2018_griC,EBV,EBV_err,dm15,dm15_err,MV,MV_err,bv_source_w09,Type,foundation,csp,w09,vel,vel_err,vel_source,bv_source,z,z_err,z_wang09,c_sncosmo,c_err_sncosmo,chisq,mwebv_sncosmo,mwebv_err,success,t0,t0_err,x0_sncosmo,x0_err_sncosmo,x1_sncosmo,x1_err_sncosmo,z_cosana,Name_sncosmo,Survey_sncosmo,bands,bands2,Name_upper,Name_snana,zHEL,zCMB,zHD,mwebv_snana,SNRMAX1,SNRMAX2,PKMJD,PKMJDERR,x1_snana,x1_err_snana,c_snana,c_err_snana,mB,mBERR,x0_snana,x0_err_snana,FITCHI2,x0_offset,Survey_snana,first_fitmjd,mb_snana,mb_err_snana,mb_sncosmo,mb_err_sncosmo,mass,masserr,vel_massvel,velerr,subsample,symbol,MU,MURES,MUERR
0,139.0,254.0,2016afk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,-12.624788,0.252733,local,NaN,0.045605,0.0,NaN,0.034624,0.035000,63.319883,0.049834,0.0,1.0,57440.76488,0.271972,0.001432,0.000053,-0.255388,0.199902,0.045605,2016afk,Foundation,gr,girz,SN2016AFK,SN2016AFK,0.04561,0.04561,0.04561,0.049834,70.51399,40.70861,57440.773,0.249,-0.272561,0.19585,0.034596,0.034857,17.47869,0.041424,0.001830,0.000070,64.08495,0.001427,Foundation,57437.395,17.478688,0.041405,17.744804,0.040399,9.774000,0.158451,-12.624788,0.252733,Foundation,o,36.6563,0.1193,0.1526
1,162.0,323.0,2016coj,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,-11.950700,0.221880,local,NaN,0.004523,0.0,NaN,-0.008069,0.032242,46.720368,0.016303,0.0,1.0,57548.18039,0.089282,0.102614,0.003310,-1.480003,0.069457,0.004523,2016coj,Foundation,gr,girz,SN2016COJ,SN2016COJ,0.00452,0.00452,0.00452,0.016303,722.75903,640.24072,57548.156,0.089,-1.512150,0.06997,-0.011431,0.032360,12.83632,0.035094,0.131670,0.004256,46.23712,0.102680,Foundation,57538.359,12.836283,0.035093,13.106986,0.035022,10.834473,0.158451,-11.950700,0.221880,Foundation,o,NaN,NaN,NaN
2,142.0,267.0,2016cor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,-9.631224,0.231824,local,NaN,0.049543,0.0,NaN,0.144141,0.033873,29.573831,0.026638,0.0,1.0,57541.71586,0.243029,0.000807,0.000028,-0.549011,0.283973,0.049543,2016cor,Foundation,gr,girz,AT2016COR,AT2016COR,0.04954,0.04954,0.04954,0.026638,102.44293,47.34186,57541.734,0.244,-0.572690,0.28299,0.143860,0.033755,18.10193,0.038142,0.001031,0.000036,29.06979,0.000804,Foundation,57540.418,18.101927,0.038124,18.367629,0.037934,10.289000,0.158451,-9.631224,0.231824,Foundation,o,36.9206,0.1978,0.1548
3,112.0,145.0,2016cvv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,-12.264770,0.566730,local,NaN,0.044571,0.0,NaN,0.066982,0.037864,26.972712,0.088615,0.0,1.0,57558.37593,0.584809,0.001835,0.000077,1.368950,0.276327,0.044571,2016cvv,Foundation,gr,girz,AT2016CVV,AT2016CVV,0.04457,0.04457,0.04457,0.088616,64.64679,47.70187,57558.336,0.623,1.366250,0.28502,0.067344,0.038454,17.20936,0.048920,0.002346,0.000106,26.20644,0.001829,Foundation,57559.465,17.209360,0.048920,17.475765,0.045258,11.104834,0.158451,-12.264770,0.566730,Foundation,o,36.4811,-0.0042,0.1655
4,125.0,190.0,2016esh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,-10.448679,0.224107,local,NaN,0.044573,0.0,NaN,-0.095078,0.037691,22.167443,0.030740,0.0,1.0,57613.30235,0.199482,0.002585,0.000102,-1.294499,0.250110,0.044573,2016esh,Foundation,gr,girz,SN2016ESH,SN2016ESH,0.04457,0.04457,0.04457,0.030740,84.47298,75.70850,57613.289,0.204,-1.320150,0.25124,-0.097388,0.037792,16.83376,0.043377,0.003315,0.000132,20.95390,0.002585,Foundation,57609.289,16.833762,0.043377,17.103778,0.042649,9.937000,0.158451,-10.448679,0.224107,Foundation,o,36.2372,-0.2481,0.1626


Reading in a file that has which velocity measurement was chosen for each object. We also want to ensure that we are only including objects with a reasonable velocity measurement from this file, which contains all measurements, so abnormally low or high velocity measurements are excluded.

In [12]:
vel_key = pd.read_csv('data/vel_choice_key.csv',comment='#')
vel_key['vel_final'] = np.nan
vel_key['vel_final_err'] = np.nan
vel_key['z'] = np.nan
for i in vel_key.index:
    name = vel_key.loc[i,'Name']
    try:
        vel_key.loc[i,'vel_final'] = master.loc[master[master.Name == name].index, 'vel'].values[0]
        vel_key.loc[i,'vel_final_err'] = master.loc[master[master.Name == name].index, 'vel_err'].values[0]
        vel_key.loc[i,'z'] = master.loc[master[master.Name == name].index, 'z'].values[0]
    except IndexError:
        continue
        
vel_key.loc[(vel_key.vel_dettman > -6) | (vel_key.vel_dettman < -20),'vel_dettman_err'] = np.nan
vel_key.loc[(vel_key.vel_dettman > -6) | (vel_key.vel_dettman < -20),'vel_dettman'] = np.nan
vel_key.loc[(vel_key.vel_siebert > -6) | (vel_key.vel_siebert < -20),'vel_siebert_err'] = np.nan
vel_key.loc[(vel_key.vel_siebert > -6) | (vel_key.vel_siebert < -20),'vel_siebert'] = np.nan
vel_key.dropna(subset=['vel_dettman','vel_siebert'],how='all',inplace=True)

The velocity measurements we have recorded have previously been phased to maximum light. This step will remove the effects of this phasing on both the measurement and the error.

In [13]:
def unphase(vel0,phase):
    return vel0*(1-0.0322*phase) - 0.285*phase
def unerr(err):
    return np.sqrt(err**2.0 - 0.22**2.0)
vel_key['vel_dettman_unphased'] = unphase(vel_key['vel_dettman'],vel_key['phase'])
vel_key['vel_dettman_err_unphased'] = unerr(vel_key['vel_dettman_err'])
vel_key['vel_siebert_unphased'] = unphase(vel_key['vel_siebert'],vel_key['phase'])
vel_key['vel_siebert_err_unphased'] = unerr(vel_key['vel_siebert_err'])
vel_key['vel_final_unphased'] = unphase(vel_key['vel_final'],vel_key['phase'])
vel_key['vel_final_err_unphased'] = unerr(vel_key['vel_final_err'])

In [14]:
vel_key_badphase = vel_key[(vel_key.phase < -6) | (vel_key.phase > 10)]
vel_key = vel_key[(vel_key.phase > -6) & (vel_key.phase < 10)]
vel_key.dropna(subset=['vel_final'],inplace=True)
vel_key.sort_values(by='Name',inplace=True)
vel_key_badphase.sort_values(by='Name',inplace=True)
vel_key.head()

,Unnamed: 0,Name,phase,choice,vel_dettman,vel_dettman_err,vel_siebert,vel_siebert_err,vel_final,vel_final_err,z,vel_dettman_unphased,vel_dettman_err_unphased,vel_siebert_unphased,vel_siebert_err_unphased,vel_final_unphased,vel_final_err_unphased
0,0,2016afk,-5.211335,average,-12.460428,0.222427,-12.789148,0.250599,-12.624788,0.252733,0.045605,-13.066120,0.032770,-13.45,0.12,-13.258060,0.124394
129,130,2016coj,-1.107989,dettman,-11.950700,0.221880,NaN,NaN,-11.950700,0.221880,0.004523,-12.061291,0.028822,NaN,NaN,-12.061291,0.028822
8,8,2016cor,-2.608754,same,-9.631224,0.231824,-9.477377,0.228035,-9.631224,0.231824,0.049543,-9.696770,0.073090,-9.53,0.06,-9.696770,0.073090
130,131,2016cvv,1.802656,dettman,-12.264770,0.566730,NaN,NaN,-12.264770,0.566730,0.044571,-12.066612,0.522286,NaN,NaN,-12.066612,0.522286
11,11,2016esh,-5.964160,same,-10.448679,0.224107,-10.653772,0.223607,-10.448679,0.224107,0.044573,-10.755520,0.042710,-11.00,0.04,-10.755520,0.042710


Collating a list of which points appear in which plots. There are currently 8 plots which are as follows:

Figure 1: A comparison between the B-V offset from FK11 and the SALT2 version of the same plot

Figure 2: Offsets for all three samples as well as a combined sample

Figure 3: BVri vs gri and BV vs ri color plots

Figure 4: Peak color plots

Figure 5: Average spectra plot (no need to really list this one)

Figure 6: Mass vs. velocity plot

Figure 7: Velocity and mass vs c and $x_1$

Figure 8: Siebert Plot

Figure 9: CDFs

In [15]:
master_snana = master.rename(columns={'c_snana':'c','x1_snana':'x1'})
names_fig1 = pd.read_csv('data/w09CompName.txt').Name.to_numpy()
print('1: '+str(len(names_fig1)))
names_fig2 = cosmo_cuts(master_snana)['Name'].to_numpy()
print('2: '+str(len(names_fig2)))
names_fig3a = pd.read_csv('data/gri_bvriNames.txt').Name.to_numpy()
print('3a: '+str(len(names_fig3a)))
names_fig3b = pd.read_csv('data/bv_riNames.txt').Name.to_numpy()
print('3b: '+str(len(names_fig3b)))
names_fig4 = pd.read_csv('data/colorColorNames.txt').Name.to_numpy()
print('4: '+str(len(names_fig4)))
names_fig6 = massvel[(~np.isnan(massvel.mass)) & (~np.isnan(massvel.vel)) & (np.abs(massvel.vel) < 99) & (massvel.Name != 'asassn-15mi') & (massvel.Name != '2007a') & (massvel.Name != '1997e')].Name.to_numpy()
print('6: '+str(len(names_fig6)))
names_fig7 = master_snana[(~np.isnan(master_snana['c'])) & (~np.isnan(master_snana['x1'])) & (np.isin(master_snana.Name,names_fig6))].Name.to_numpy()
print('7: '+str(len(names_fig7)))
names_fig8 = master_found_cosmo[(~np.isnan(master_found_cosmo.MURES)) & (np.isin(master_found_cosmo.Name,names_fig6))].Name.to_numpy()
print('8: '+str(len(names_fig8)))
names_fig9a = pd.read_csv('data/cdf_standardNames.txt').Name.to_numpy()
print('9a: '+str(len(names_fig9a)))
names_fig9b = pd.read_csv('data/cdf_x1Names.txt').Name.to_numpy()
print('9b: '+str(len(names_fig9b)))
names_fig9c = pd.read_csv('data/cdf_mass95Names.txt').Name.to_numpy()
print('9c: '+str(len(names_fig9c)))

1: 88
2: 215
3a: 74
3b: 162
4: 136
6: 222
7: 198
8: 64
9a: 231
9b: 197
9c: 176


Removing any objects which appear in no figures

In [16]:
for i in master_found_cosmo.index:
    name = master_found_cosmo.loc[i,'Name']
    if (name not in names_fig1) and (name not in names_fig2) and (name not in names_fig3a) and (name not in names_fig3b) and (name not in names_fig4) and (name not in names_fig6) and (name not in names_fig7) and (name not in names_fig8) and (name not in names_fig9a) and (name not in names_fig9b) and (name not in names_fig9c):
        print(master_found_cosmo[master_found_cosmo.index == i][['Name','z','c_snana','x1_snana','vel','mass']])
        master_found_cosmo.drop(index=i,inplace=True)
        
for i in master_mass.index:
    name = master_mass.loc[i,'Name']
    if (name not in names_fig1) and (name not in names_fig2) and (name not in names_fig3a) and (name not in names_fig3b) and (name not in names_fig4) and (name not in names_fig6) and (name not in names_fig7) and (name not in names_fig8) and (name not in names_fig9a) and (name not in names_fig9b) and (name not in names_fig9c):
        print(master_mass[master_mass.index == i][['Name','z','c_snana','x1_snana','vel','mass']])
        master_mass.drop(index=i,inplace=True)

      Name         z   c_snana  x1_snana   vel  mass
88  2003hv  0.005087 -0.115961  -1.77439  99.9   NaN
       Name         z  c_snana  x1_snana        vel  mass
110  2005bc  0.012779  0.39301  -1.80522 -11.004758   NaN
       Name        z  c_snana  x1_snana  vel     mass
127  2005lu  0.03199      NaN       NaN  NaN  10.3465
       Name        z  c_snana  x1_snana  vel     mass
137  2006bd  0.02571  0.78386  -2.40305  NaN  11.0564
       Name         z  c_snana  x1_snana  vel   mass
142  2006cc  0.032666  0.35841   0.23648  NaN  10.46
       Name       z  c_snana  x1_snana  vel       mass
168  2006ot  0.0531      NaN       NaN  NaN  11.361939
       Name        z  c_snana  x1_snana  vel    mass
177  2007al  0.01211      NaN       NaN  NaN  9.6232
       Name        z   c_snana  x1_snana        vel  mass
189  2007gi  0.00482  0.086391  -2.76426 -15.894815   NaN
       Name        z  c_snana  x1_snana  vel     mass
193  2007jh  0.04077      NaN       NaN  NaN  11.1137
      Name      

In [17]:
def fixName(name):
    if ('asassn-' in name):
        new_name = re.sub('asassn','ASASSN',name)
    elif ('asassn' in name):
        new_name = re.sub('asassn','ASASSN-',name)
    elif ('ps1' in name):
        new_name = re.sub('ps','PS',name)
    elif ('atlas' in name):
        new_name = re.sub('atlas','ATLAS',name)
    elif ('psnj' in name):
        new_name = re.sub('psnj','PSNJ',name)
    elif ('gaia' in name):
        new_name = re.sub('gaia','Gaia',name)
    elif ('ptss' in name):
        new_name = re.sub('ptss','PTSS',name)
    elif ('css' in name):
        new_name = re.sub('css','CSS',name)
    elif len(name) == 5:
        new_name = name.upper()
    else:
        new_name = name
    return new_name

def figsInclude(name):
    return_string = ''
    if name in names_fig1:
        return_string = return_string + '1,'
    if name in names_fig2:
        return_string = return_string + '2,'
    if name in names_fig3a:
        return_string = return_string + '3a,'
    if name in names_fig3b:
        return_string = return_string + '3b,'
    if name in names_fig4:
        return_string = return_string + '4,'
    if name in names_fig6:
        return_string = return_string + '6,'
    if name in names_fig7:
        return_string = return_string + '7,'
    if name in names_fig8:
        return_string = return_string + '8,'
    if name in names_fig9a:
        return_string = return_string + '9a,'
    if name in names_fig9b:
        return_string = return_string + '9b,'
    if name in names_fig9c:
        return_string = return_string + '9c,'
    
    if return_string == '':
        return ''
    else:
        return return_string[0:-1]

In [18]:
def writeTable(data, cols, out_names, filename):
    with open(filename, 'w') as f:
        for i in data.index:
            out_string = ''
            for j, col in enumerate(cols):
                if col == '':
                    out_string = out_string + r'... & '
                elif col == 'figs_include':
                    out_string = out_string + figsInclude(data.loc[i,'Name']) + ' & '
                elif isinstance(data.loc[i,col],str):
                    if col == 'Name':
                        out_string = out_string + '{} & '.format(fixName(data.loc[i,col]))
                    elif col == 'source':
                        if data.loc[i,col] == 'CSP' or data.loc[i,col] == 'csp':
                            out_string = out_string + '{} & '.format(r'\text{\citet{Folatelli2013}}')
                        elif data.loc[i,col] == 'kaepora':
                            out_string = out_string + '{} & '.format(r'\text{\citet{Siebert19}}')
                        else:
                            out_string = out_string + r'\text{\citet{' + data.loc[i,col] + '}} & '
                    else:
                        out_string = out_string + '{} & '.format(data.loc[i,col])
                else:
                    if np.isnan(data.loc[i,col]):
                        out_string = out_string + r'... & '
                    else:
                        if ('c' in col) or ('z' in col):
                            out_string = out_string + '${:.3f}$ & '.format(data.loc[i,col])
                        elif ('phase' in col) and ('un' not in col):
                            out_string = out_string + '${:+.1f}$ & '.format(data.loc[i,col])
                        elif ('vel' in col):
                            if ('err' in col) and (np.abs(data.loc[i,col]) > 90):
                                out_string = out_string + r'... & '
                            elif data.loc[i,col] < -90:
                                out_string = out_string + '$<-11.8$ & '
                            elif data.loc[i,col] > 90:
                                out_string = out_string + '$>-11.8$ & '
                            else:
                                out_string = out_string + '${:.2f}$ & '.format(data.loc[i,col])
                        else:
                            out_string = out_string + '${:.2f}$ & '.format(data.loc[i,col])
            f.write(out_string[:-2] + '\\\\ \n')

In [19]:
def writeTable_mach(data, cols, out_names, filename):
    with open(filename, 'w') as f:
        for i in data.index:
            out_string = ''
            for j, col in enumerate(cols):
                if col == '':
                    out_string = out_string + 'nan,'
                elif isinstance(data.loc[i,col],str):
                    if col == 'Name':
                        out_string = out_string + '{},'.format(fixName(data.loc[i,col]))
                    else:
                        out_string = out_string + '{},'.format(data.loc[i,col])
                else:
                    if np.isnan(data.loc[i,col]):
                        out_string = out_string + 'nan,'
                    else:
                        if ('c' in col) or ('z' in col):
                            out_string = out_string + '{:.3f},'.format(data.loc[i,col])
                        elif ('phase' in col) and ('un' not in col):
                            out_string = out_string + '{:+.1f},'.format(data.loc[i,col])
                        elif ('vel' in col):
                            if ('err' in col) and (np.abs(data.loc[i,col]) > 90):
                                out_string = out_string + r'nan,'
                            elif data.loc[i,col] < -90:
                                out_string = out_string + '<-11.8,'
                            elif data.loc[i,col] > 90:
                                out_string = out_string + '>-11.8,'
                            else:
                                out_string = out_string + '{:.2f},'.format(data.loc[i,col])
                        else:
                            out_string = out_string + '{:.2f},'.format(data.loc[i,col])
            f.write(out_string[:-2] + ' \n')

In [20]:
writeTable(master_found_cosmo, 
           ['Name','z','vel','vel_err','c_snana','c_err_snana','x1_snana','x1_err_snana','PKMJD','PKMJDERR',
            'mb_snana','mb_err_snana','MURES','MUERR','mass','masserr','figs_include'], 
           ['Name','z','Velocity [1000 km/s]','e_Velocity','c','e_c','x1','e_x1','t0','e_t0','$m_B$','e_$m_B$',
            r'$\mu_{res}$',r'e_$\mu$','Host Mass','e_Host Mass','Figs'],
           'tables/found_snana.tex')

In [21]:
writeTable(master_mass[master_mass.csp == True], 
           ['Name','z','vel','vel_err','c_snana','c_err_snana','x1_snana','x1_err_snana','PKMJD','PKMJDERR',
            'mb_snana','mb_err_snana','mass','masserr','figs_include'], 
           ['Name','z','Velocity [1000 km/s]','e_Velocity','c','e_c','x1','e_x1','t0','e_t0','$m_B$','e_$m_B$',
            'Host Mass','e_Host Mass','Figs'],
           'tables/csp_snana.tex')

In [22]:
writeTable(master_mass[master_mass.w09 == True], 
           ['Name','z','vel','vel_err','c_snana','c_err_snana','x1_snana','x1_err_snana','PKMJD','PKMJDERR',
            'mb_snana','mb_err_snana','mass','masserr','figs_include'], 
           ['Name','z','Velocity [1000 km/s]','e_Velocity','c','e_c','x1','e_x1','t0','e_t0','$m_B$','e_$m_B$',
            'Host Mass','e_Host Mass','Figs'],
           'tables/w09_snana.tex')

In [23]:
writeTable(vel_key, ['Name','phase','vel_dettman_unphased','vel_dettman_err_unphased','vel_siebert_unphased',
                     'vel_siebert_err_unphased','vel_final_unphased','vel_final_err_unphased',
                     'vel_final','vel_final_err'],
           ['Name','Phase','Unphased Velocity Gaussian Fit','Unphased e_Velocity Gaussian Fit',
            'Unphased Velocity Smooth Fit','Unphased e_Velocity Smooth Fit','Unphased Final Velocity',
            'Unphased e_Final Velocity','Final Velocity','e_Final Velocity'],
          'tables/vel_fits.tex')

In [24]:
writeTable(vel_key_badphase, ['Name','phase','vel_dettman_unphased','vel_dettman_err_unphased',
                              'vel_siebert_unphased','vel_siebert_err_unphased','','','',''],
           ['Name','Phase','Velocity Gaussian Fit','e_Velocity Gaussian Fit',
            'Velocity Smooth Fit','e_Velocity Smooth Fit'],
          'tables/vel_fits_badphase.tex')

In [25]:
new_w09_csp_vel = pd.read_csv('data/new_w09_CSP_vels.csv')
w09csp_vel = new_w09_csp_vel[~np.isnan(new_w09_csp_vel.new_vel)]
w09csp_vel_badphase = new_w09_csp_vel[np.isnan(new_w09_csp_vel.new_vel)]

writeTable(w09csp_vel, ['Name','phase','kyle_vel','kyle_vel_err','matt_vel','matt_vel_err',
                        'new_vel_unphased','new_vel_err_unphased','new_vel','new_vel_err','source'],
           ['Name','Phase','Unphased Velocity Gaussian Fit','Unphased e_Velocity Gaussian Fit',
            'Unphased Velocity Smooth Fit','Unphased e_Velocity Smooth Fit','Unphased Final Velocity',
            'Unphased e_Final Velocity','Final Velocity','e_Final Velocity','Source'],
           'tables/vel_fits_csp_w09.tex')

writeTable(w09csp_vel_badphase, ['Name','phase','kyle_vel','kyle_vel_err','matt_vel','matt_vel_err',
                                 'new_vel_unphased','new_vel_err_unphased','new_vel','new_vel_err','source'],
           ['Name','Phase','Unphased Velocity Gaussian Fit','Unphased e_Velocity Gaussian Fit',
            'Unphased Velocity Smooth Fit','Unphased e_Velocity Smooth Fit','Unphased Final Velocity',
            'Unphased e_Final Velocity','Final Velocity','e_Final Velocity','Source'],
           'tables/vel_fits_csp_w09_badphase.tex')

In [27]:
w09csp_vel[w09csp_vel.Name == '2004s']

,Unnamed: 0,Unnamed: 0.1,Name,matt_vel,matt_vel_err,kyle_vel,kyle_vel_err,phase,kyle_pvel,kyle_pvel_err,matt_pvel,matt_pvel_err,choice,source,new_vel,new_vel_err,new_vel_unphased,new_vel_err_unphased
194,300,300,2004s,-9.6,0.08,-9.98,0.1,-0.18882,-9.973177,0.241661,-9.595473,0.234094,kyle,Krisciunas2007,-9.973177,0.241661,-9.98,0.1
